In [1]:
import itertools
from collections import Counter

In [2]:
N = ToricLattice(7)

K = N([-3, 1, 1, 1, 1, 1, 1]) # canonical divisor
L = N([1, 0, 0, 0, 0, 0, 0])

Line = dict()
for i,e in enumerate(N.basis()):
    if i>0:
        Line[f'E{i}'] = e
        Line[f'Q{i}'] = -K-L+e

for i,j in itertools.combinations(range(1,7), 2):
    Line[f'L{i}{j}'] = L-Line[f'E{i}']-Line[f'E{j}']

Line = dict(sorted(Line.items()))
minus_one_curves = list(Line.values())

In [3]:
_dot_matrix = diagonal_matrix([1]+6*[-1])
dot = lambda a, b: a*_dot_matrix*b
gram_matrix = lambda rays: matrix([[dot(a,b) for a in rays] for b in rays])
curves_disjoint_with = lambda curves: [c for c in minus_one_curves if all(dot(c,c2)==0 for c2 in curves)]
P2_line_class = lambda exceptional_curves : N([i/3 for i in (-K + sum(exceptional_curves)) ])
NE = Cone(minus_one_curves)
Ample = Cone([N(_dot_matrix*ray) for ray in NE.dual().rays()])
is_covered_by = lambda cone1, cone2: all(cone2.contains(ray) for ray in cone1.rays()) and cone2.interior_contains(sum(cone1.rays()))

In [4]:
double_intersections = [(a,b) for a,b in itertools.combinations(minus_one_curves, 2) if dot(a,b)==1]
Eckardt_points = \
[[f'E{i}',f'Q{j}',f'L{min(i,j)}{max(i,j)}'] for i,j in itertools.permutations([1,2,3],int(2))] + \
[[f'E{i}',f'Q{j}',f'L{min(i,j)}{max(i,j)}'] for i,j in itertools.permutations([4,5,6],int(2))] + \
[[f'L1{i}',f'L2{j}',f'L3{k}'] for i,j,k in itertools.permutations([4,5,6])]
#Counter([p for tr in Eckardt_points for p in tr])
Eckardt_points = [tuple(Line[i] for i in triple) for triple in Eckardt_points]
#triple_intersections = [(a,b,c) for a,b,c in itertools.combinations(minus_one_curves, 3) if dot(a,b)+dot(b,c)+dot(a,c)==3]


In [5]:
def independent_sets(vectors, size = 6):
    if size == 0:
        yield []
    for i, v in enumerate(vectors):
        orthogonals = [v2 for v2 in vectors[i+1:] if dot(v, v2)==0]
        for subset in independent_sets(orthogonals, size-1):
            yield subset + [v]

def cylinder_generator_P1(E):
    L = P2_line_class(E)
    for i,j in itertools.permutations(range(len(E)),int(2)):
        complement = [k for k in range(len(E)) if (k!=i) and (k!=j)]
        contracted_curves = [E[i] for i in complement] + [L-E[i]-E[j]]
        conic = -K-L+E[i]

        triple_points = [p for p in Eckardt_points if conic in p]
        tangents = [l for p in triple_points for l in p if (l not in contracted_curves) and (l!=conic)]
        if False:
            print('tangents ',tangents)
            print('E ',E)
            print('Eckardt ',triple_points)
            print('contracted ',contracted_curves)
            print('Q ',conic)
            print('i,j ',i,j)
            raise Error
        if len(tangents)>2:
            continue
        q = lambda T: [e for e in contracted_curves if dot(e,T)==1][0]
        things = [{
            'T': T,
            'q': q(T),
            'R': L-E[j]-q(T),
        } for T in tangents]
        points = [
            [things[0]['T'],things[1]['R'],],
            [things[1]['T'],things[0]['R'],],
        ]            
        curves = [E[k] for k in complement]+[L-E[i]-E[j], conic]
        yield {
            'rays': curves + [L-E[i],L-E[j]],
            'curves': curves,
            'points': points,
        }


def cylinders_of_type(cylinder_generator_list):
    for cylinder_generator in cylinder_generator_list:
        for blowdown in independent_sets(minus_one_curves):
            yield from cylinder_generator(blowdown)



cylinders = [dict(cylinder,**{'index': i}) for i, cylinder in enumerate(cylinders_of_type([cylinder_generator_P1]))]


def minus_one_curves_in_complement(collection):
    return [curve for curve in minus_one_curves if all(curve in cylinder['rays'] for cylinder in collection)]



In [6]:
cylinders[10]

{'rays': [N(0, 0, 0, 0, 0, 1, 0),
  N(0, 0, 0, 1, 0, 0, 0),
  N(0, 0, 1, 0, 0, 0, 0),
  N(0, 1, 0, 0, 0, 0, 0),
  N(1, 0, 0, 0, -1, 0, -1),
  N(2, -1, -1, -1, 0, -1, -1),
  N(1, 0, 0, 0, -1, 0, 0),
  N(1, 0, 0, 0, 0, 0, -1)],
 'curves': [N(0, 0, 0, 0, 0, 1, 0),
  N(0, 0, 0, 1, 0, 0, 0),
  N(0, 0, 1, 0, 0, 0, 0),
  N(0, 1, 0, 0, 0, 0, 0),
  N(1, 0, 0, 0, -1, 0, -1),
  N(2, -1, -1, -1, 0, -1, -1)],
 'points': [[N(1, 0, 0, 0, -1, -1, 0), N(0, 0, 0, 0, 1, 0, 0)],
  [N(0, 0, 0, 0, 0, 0, 1), N(1, 0, 0, 0, 0, -1, -1)]],
 'index': 10}

In [7]:

def Pol(collection):
    result = Cone([],lattice=N.dual()).dual() # ambient space
    for cylinder in collection:
        result = result.intersection(cylinder['Pol'])
    return result
    
# forbidden cone of a collection of cylinders
def Forb(collection):
    return Cone(minus_one_curves_in_complement(collection),lattice=N)

for u in cylinders:
    u['Pol'] = Cone(u['rays'])
    #u['Forb'] = Forb([u])


In [8]:

def is_polar_wrt(cylinder, cones):
    return all(is_covered_by(cone,cylinder['Pol']) for cone in cones)

def points_coincide(point1, point2):
    return all(curve in point2 for curve in point1) and all(curve in point1 for curve in point2)

intersection_point_in_cylinder = lambda intersection, cylinder: all(curve not in cylinder['rays'] for curve in intersection) \
                                                            and all(not points_coincide(intersection,p) for p in cylinder['points'])

for u in cylinders:
    u['double'] = [intersection for intersection in double_intersections if not intersection_point_in_cylinder(intersection, u)]
    u['Eckardt'] = [intersection for intersection in Eckardt_points if not intersection_point_in_cylinder(intersection, u)]

intersection_point_in_cylinders = lambda intersection, cylinders: any(intersection_point_in_cylinder(intersection, c) for c in cylinders)
def complement_points(collection):
    return {
        'double': [intersection for intersection in double_intersections if not intersection_point_in_cylinders(intersection, collection)],
        'Eckardt': [intersection for intersection in Eckardt_points if not intersection_point_in_cylinders(intersection, collection)],
    }


In [9]:
cylinders[0]

{'rays': [N(0, 0, 0, 0, 1, 0, 0),
  N(0, 0, 0, 1, 0, 0, 0),
  N(0, 0, 1, 0, 0, 0, 0),
  N(0, 1, 0, 0, 0, 0, 0),
  N(1, 0, 0, 0, 0, -1, -1),
  N(2, -1, -1, -1, -1, -1, 0),
  N(1, 0, 0, 0, 0, 0, -1),
  N(1, 0, 0, 0, 0, -1, 0)],
 'curves': [N(0, 0, 0, 0, 1, 0, 0),
  N(0, 0, 0, 1, 0, 0, 0),
  N(0, 0, 1, 0, 0, 0, 0),
  N(0, 1, 0, 0, 0, 0, 0),
  N(1, 0, 0, 0, 0, -1, -1),
  N(2, -1, -1, -1, -1, -1, 0)],
 'points': [[N(1, 0, 0, 0, -1, 0, -1), N(0, 0, 0, 0, 0, 0, 1)],
  [N(0, 0, 0, 0, 0, 1, 0), N(1, 0, 0, 0, -1, -1, 0)]],
 'index': 0,
 'Pol': 7-d cone in 7-d lattice N,
 'double': [(N(0, 1, 0, 0, 0, 0, 0), N(1, -1, -1, 0, 0, 0, 0)),
  (N(0, 1, 0, 0, 0, 0, 0), N(1, -1, 0, -1, 0, 0, 0)),
  (N(0, 1, 0, 0, 0, 0, 0), N(1, -1, 0, 0, -1, 0, 0)),
  (N(0, 1, 0, 0, 0, 0, 0), N(1, -1, 0, 0, 0, -1, 0)),
  (N(0, 1, 0, 0, 0, 0, 0), N(1, -1, 0, 0, 0, 0, -1)),
  (N(0, 1, 0, 0, 0, 0, 0), N(2, -1, 0, -1, -1, -1, -1)),
  (N(0, 1, 0, 0, 0, 0, 0), N(2, -1, -1, 0, -1, -1, -1)),
  (N(0, 1, 0, 0, 0, 0, 0), N(2, -1, -1,

In [10]:

def point_eliminating_cylinder(pool, U, seed=123456):
    polarity = Pol(U).intersection(Ample) if len(U)>0 else Ample
    double_points = complement_points(U)['double']
    print('double points in complement: ', len(double_points))
    polarity_agrees = lambda cone: is_covered_by(cone, Ample) and cone.codim()==0
    candidates = [c for c in pool if polarity_agrees(c['Pol'].intersection(polarity))]
    print('candidates: ', len(candidates))
    if len(candidates) == 0:
        return None
    candidates_with_counts = [(c, len([point for point in double_points if intersection_point_in_cylinder(point, c)])) for c in candidates]
    max_count = max([c[1] for c in candidates_with_counts])
    print('points eliminated: ', max_count)
    if max_count == 0:
        return None
    best_candidates = [candidate for candidate, count in candidates_with_counts if count == max_count]
    print('best candidates: ', len(best_candidates))
    print('returning ', best_candidates[0])
    return best_candidates[0] #[seed%len(best_candidates)]

def eliminate_points(pool=cylinders, collection=[]):
    new_collection = collection[:]
    while True:
        new_cylinder = point_eliminating_cylinder(pool, new_collection)
        #print('new cylinder', new_cylinder)
        if new_cylinder == None:
            break
        new_collection.append(new_cylinder)
        print('New collection length:\n', len(new_collection))
    return new_collection

def eliminate_cylinders(collection):
    if len(complement_points(collection)['double'])!=0:
        print('this collection does not cover all double points')
        return collection
    print('deleting cylinders, remaining:')
    while True:
        for i in range(len(collection)):
            collection_without_cylinder = collection[:i] + collection[i+1:]
            if len(complement_points(collection_without_cylinder)['double'])==0:
                del collection[i]
                print(len(collection), end=' ')
                break
        else:
            print('no cylinders to delete')
            break
    return collection


In [11]:
collection = eliminate_points()
shorter_collection = eliminate_cylinders(collection)
Pol(shorter_collection)

double points in complement:  135
candidates:  2160
points eliminated:  78
best candidates:  2160
returning  {'rays': [N(0, 0, 0, 0, 1, 0, 0), N(0, 0, 0, 1, 0, 0, 0), N(0, 0, 1, 0, 0, 0, 0), N(0, 1, 0, 0, 0, 0, 0), N(1, 0, 0, 0, 0, -1, -1), N(2, -1, -1, -1, -1, -1, 0), N(1, 0, 0, 0, 0, 0, -1), N(1, 0, 0, 0, 0, -1, 0)], 'curves': [N(0, 0, 0, 0, 1, 0, 0), N(0, 0, 0, 1, 0, 0, 0), N(0, 0, 1, 0, 0, 0, 0), N(0, 1, 0, 0, 0, 0, 0), N(1, 0, 0, 0, 0, -1, -1), N(2, -1, -1, -1, -1, -1, 0)], 'points': [[N(1, 0, 0, 0, -1, 0, -1), N(0, 0, 0, 0, 0, 0, 1)], [N(0, 0, 0, 0, 0, 1, 0), N(1, 0, 0, 0, -1, -1, 0)]], 'index': 0, 'Pol': 7-d cone in 7-d lattice N, 'double': [(N(0, 1, 0, 0, 0, 0, 0), N(1, -1, -1, 0, 0, 0, 0)), (N(0, 1, 0, 0, 0, 0, 0), N(1, -1, 0, -1, 0, 0, 0)), (N(0, 1, 0, 0, 0, 0, 0), N(1, -1, 0, 0, -1, 0, 0)), (N(0, 1, 0, 0, 0, 0, 0), N(1, -1, 0, 0, 0, -1, 0)), (N(0, 1, 0, 0, 0, 0, 0), N(1, -1, 0, 0, 0, 0, -1)), (N(0, 1, 0, 0, 0, 0, 0), N(2, -1, 0, -1, -1, -1, -1)), (N(0, 1, 0, 0, 0, 0, 0), N(2

7-d cone in 7-d lattice N

In [12]:
def line_name(ray): 
    candidates = [i for i,r in Line.items() if r==ray]
    if len(candidates)>0:
        return candidates[0]
    else:
        return '__'

for c in collection:
    print('\n',c['index'])
    for i, ray in enumerate(c['rays']):
        print(i+1,line_name(ray),ray)
    print([[line_name(l) for l in p] for p in c['points']])



 0
1 E4 N(0, 0, 0, 0, 1, 0, 0)
2 E3 N(0, 0, 0, 1, 0, 0, 0)
3 E2 N(0, 0, 1, 0, 0, 0, 0)
4 E1 N(0, 1, 0, 0, 0, 0, 0)
5 L56 N(1, 0, 0, 0, 0, -1, -1)
6 Q6 N(2, -1, -1, -1, -1, -1, 0)
7 __ N(1, 0, 0, 0, 0, 0, -1)
8 __ N(1, 0, 0, 0, 0, -1, 0)
[['L46', 'E6'], ['E5', 'L45']]

 297
1 L35 N(1, 0, 0, -1, 0, -1, 0)
2 L25 N(1, 0, -1, 0, 0, -1, 0)
3 E6 N(0, 0, 0, 0, 0, 0, 1)
4 E4 N(0, 0, 0, 0, 1, 0, 0)
5 L15 N(1, -1, 0, 0, 0, -1, 0)
6 L46 N(1, 0, 0, 0, -1, 0, -1)
7 __ N(2, -1, -1, -1, 0, -1, 0)
8 __ N(1, 0, 0, 0, 0, -1, 0)
[['Q6', 'L56'], ['Q4', 'L45']]

 263
1 L36 N(1, 0, 0, -1, 0, 0, -1)
2 L26 N(1, 0, -1, 0, 0, 0, -1)
3 L23 N(1, 0, -1, -1, 0, 0, 0)
4 E1 N(0, 1, 0, 0, 0, 0, 0)
5 Q1 N(2, 0, -1, -1, -1, -1, -1)
6 L15 N(1, -1, 0, 0, 0, -1, 0)
7 __ N(2, 0, -1, -1, -1, 0, -1)
8 __ N(2, 0, -1, -1, 0, -1, -1)
[['L24', 'L35'], ['L34', 'L25']]

 115
1 L36 N(1, 0, 0, -1, 0, 0, -1)
2 L34 N(1, 0, 0, -1, -1, 0, 0)
3 E5 N(0, 0, 0, 0, 0, 1, 0)
4 E2 N(0, 0, 1, 0, 0, 0, 0)
5 L13 N(1, -1, 0, -1, 0, 0, 0)
6 L25 N(1,

In [13]:
Pol(collection).intersection(Ample).rays()

N( 2, -1, -1, -1,  0, -1,  0),
N( 1,  0,  0,  0,  0, -1,  0),
N( 3,  0,  0, -1,  0, -2, -1),
N( 3, -1, -1, -1, -1, -1, -1),
N( 3,  0, -1, -1,  0, -2, -1),
N( 8,  0, -2, -3, -1, -5, -2),
N( 6,  0, -2, -2, -1, -4, -1),
N( 5,  0, -2, -1,  0, -3, -1),
N( 9,  0, -2, -3,  0, -5, -3),
N( 3,  0, -1, -1,  0, -2,  0),
N(11,  0, -2, -4, -1, -6, -3),
N( 7,  0, -2, -2, -1, -4, -1),
N( 5,  0, -1, -2,  0, -3, -1),
N( 8, -1, -3, -2, -1, -5, -2),
N(13, -1, -5, -4, -2, -8, -3)
in 7-d lattice N

In [14]:
def cylinder_generator_P2(E):
    '''
    E is a list of exceptional curves, and e is one of them
    returns 
    1. list of rays of the cone of divisors H such that the cylinder (P^2 minus conic through E-e and tangent line through e) is H-polar
    2. list of (-1)-curves lying in the complement of the cylinder (or a union of cylinders if many)
    '''
    for e in E:
        tangent = P2_line_class(E) - e
        conic = - K - tangent
        curves = E+[conic]
        yield {
            'rays': curves + [tangent],
            'curves': curves,
            'points': [],
        }

cylinders_P2 = [dict(cylinder,**{'index': i}) for i, cylinder in enumerate(cylinders_of_type([cylinder_generator_P2]))]

for u in cylinders_P2:
    u['Pol'] = Cone(u['rays'])
    
collection_P2 = eliminate_points(pool=cylinders_P2)
shorter_collection_P2 = eliminate_cylinders(collection_P2)

for c in shorter_collection_P2:
    print('\n',c['index'])
    for i, ray in enumerate(c['rays']):
        print(i+1,line_name(ray),ray)
    print([[line_name(l) for l in p] for p in c['points']])
Pol(shorter_collection_P2).rays()


double points in complement:  135
candidates:  432
points eliminated:  70
best candidates:  432
returning  {'rays': [N(0, 0, 0, 0, 0, 0, 1), N(0, 0, 0, 0, 0, 1, 0), N(0, 0, 0, 0, 1, 0, 0), N(0, 0, 0, 1, 0, 0, 0), N(0, 0, 1, 0, 0, 0, 0), N(0, 1, 0, 0, 0, 0, 0), N(2, -1, -1, -1, -1, -1, 0), N(1, 0, 0, 0, 0, 0, -1)], 'curves': [N(0, 0, 0, 0, 0, 0, 1), N(0, 0, 0, 0, 0, 1, 0), N(0, 0, 0, 0, 1, 0, 0), N(0, 0, 0, 1, 0, 0, 0), N(0, 0, 1, 0, 0, 0, 0), N(0, 1, 0, 0, 0, 0, 0), N(2, -1, -1, -1, -1, -1, 0)], 'points': [], 'index': 0, 'Pol': 7-d cone in 7-d lattice N}
New collection length:
 1
double points in complement:  65
candidates:  172
points eliminated:  37
best candidates:  20
returning  {'rays': [N(2, -1, -1, -1, -1, -1, 0), N(2, -1, -1, -1, -1, 0, -1), N(2, -1, -1, -1, 0, -1, -1), N(1, 0, -1, -1, 0, 0, 0), N(1, -1, 0, -1, 0, 0, 0), N(1, -1, -1, 0, 0, 0, 0), N(1, 0, 0, 0, -1, -1, 0), N(2, -1, -1, -1, 0, 0, -1)], 'curves': [N(2, -1, -1, -1, -1, -1, 0), N(2, -1, -1, -1, -1, 0, -1), N(2, -1, 

N( 2, -1, -1, -1, -1, -1,  0),
N( 4, -1, -1, -2, -2, -2,  0),
N( 3, -1, -1, -1, -1, -1, -1),
N( 9, -4, -3, -3, -4, -4, -1),
N( 9, -3, -4, -3, -4, -4, -1),
N(11, -5, -4, -4, -5, -4, -1),
N(11, -4, -5, -4, -5, -4, -1),
N(13, -5, -5, -5, -5, -6, -1),
N( 7, -3, -3, -3, -3, -2, -1),
N( 5, -2, -2, -2, -2, -2,  0)
in 7-d lattice N

In [15]:
for u in shorter_collection:
    for l1 in u['points'][0]:
        for l2 in u['points'][1]:
            disjoint = [ray for ray in u['rays'][:5] if dot(ray,l1)==0 and dot(ray,l2)==0]
            if len(disjoint)==4:
                print(u['index'])
                print('T,R',l1,l2)
                print('T', [l for l in [l1,l2] if dot(l,u['rays'][5])==1])
                print('q',[ray for ray in u['rays'][:5] if ray not in disjoint][0])
            
    print('\n')

0
T,R N(1, 0, 0, 0, -1, 0, -1) N(1, 0, 0, 0, -1, -1, 0)
T [N(1, 0, 0, 0, -1, 0, -1)]
q N(0, 0, 0, 0, 1, 0, 0)
0
T,R N(0, 0, 0, 0, 0, 0, 1) N(0, 0, 0, 0, 0, 1, 0)
T [N(0, 0, 0, 0, 0, 1, 0)]
q N(1, 0, 0, 0, 0, -1, -1)


297
T,R N(2, -1, -1, -1, -1, -1, 0) N(1, 0, 0, 0, -1, -1, 0)
T [N(2, -1, -1, -1, -1, -1, 0)]
q N(0, 0, 0, 0, 1, 0, 0)
297
T,R N(1, 0, 0, 0, 0, -1, -1) N(2, -1, -1, -1, 0, -1, -1)
T [N(2, -1, -1, -1, 0, -1, -1)]
q N(0, 0, 0, 0, 0, 0, 1)


263
T,R N(1, 0, -1, 0, -1, 0, 0) N(1, 0, -1, 0, 0, -1, 0)
T [N(1, 0, -1, 0, -1, 0, 0)]
q N(1, 0, 0, -1, 0, 0, -1)
263
T,R N(1, 0, 0, -1, 0, -1, 0) N(1, 0, 0, -1, -1, 0, 0)
T [N(1, 0, 0, -1, -1, 0, 0)]
q N(1, 0, -1, 0, 0, 0, -1)


115
T,R N(1, -1, 0, 0, -1, 0, 0) N(0, 0, 0, 0, 0, 0, 1)
T [N(1, -1, 0, 0, -1, 0, 0)]
q N(1, 0, 0, -1, 0, 0, -1)
115
T,R N(0, 0, 0, 0, 1, 0, 0) N(1, -1, 0, 0, 0, 0, -1)
T [N(1, -1, 0, 0, 0, 0, -1)]
q N(1, 0, 0, -1, -1, 0, 0)




In [16]:
p = Pol(collection)
rays = p.rays()
ray = sum([rays[i] for i in [0,2,5,7]])
p.relative_interior_contains(ray)
ray

N(16, -1, -5, -5, -1, -10, -3)

In [17]:
for r1,r2 in itertools.combinations(p.rays(),int(2)):
    if p.relative_interior_contains(r1+r2):
        print(r1+r2)

N(13, -1, -4, -4, -1, -8, -3)


In [18]:
p.relative_interior_contains(N(13, -1, -4, -4, -1, -8, -3))

True

In [35]:
for c in collection:
    print(c['Pol'],'\n')

7-d cone in 7-d lattice N 

7-d cone in 7-d lattice N 

7-d cone in 7-d lattice N 

7-d cone in 7-d lattice N 



In [34]:
H = N(13, -1, -4, -4, -1, -8, -3)
P = [c['Pol'] for c in collection]
P[3].relative_interior_contains(H)

True

In [29]:
2*H- sum(P[0].rays())

N(21, -2, -8, -8, -2, -13, -4)